## computations are represented as graphs
- nodes in the graph are called ops (operations) that perform computations on tensors
- a TensorFlow graph is a description of computations (like a list of instructions)
- to compute anything, a graph must be launched in a Session.
- A Session places graph ops onto Devices (gpu, cpu) and provides methods to execute them.
- methods return tesors produced by ops as np.arrays

## Building the graph
- start with ops that do not need any input (source ops) such as `Constant` and pass their outputs through the graph.
- graphs get built (initialized) once and run (executed) many times. 

In [1]:
import tensorflow as tf

In [2]:
# Create a Constant op that produces a 1x2 matrix. The op is added
# as a node to the default graph

# The value returned by the constructor represeents and output of the
# Constant op.
matrix1 = tf.constant([[3., 3.]]) # 1x2 matrix

matrix2 = tf.constant([[2.],
                       [2.]]) # 2x1 matrix

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

In [3]:
# If you're familiar with numpy you may think you're done but wait!
print product # where's our output value?

Tensor("MatMul:0", shape=(1, 1), dtype=float32)


In [5]:
# try again
matrix1 * matrix2 # still no output

<tf.Tensor 'mul:0' shape=(2, 2) dtype=float32>

### Remember: no computation has been done at this point. We have merely built the graph. We have to run it to get output

### Launch graph

In [6]:
sess = tf.Session()

In [7]:
result = sess.run(product)
# this causes the execution of all the ops needed to get the output of 
# `product`, two constants, then product. All ops are connected in the graph

In [8]:
print result

[[ 12.]]


In [9]:
# close session when done. Releases resources.
sess.close()

In [10]:
# if we enter a session with a "With" block 
# the session will close automatically

In [11]:
with tf.Session() as sess:
    result = sess.run(product)
    print result

[[ 12.]]


### Multi-gpu:
- if you have more than one GPU on your machine, assign ops to it explicity

- "/cpu:0": The CPU of your machine.
- "/gpu:0": The GPU of your machine, if you have one.
- "/gpu:1": The second GPU of your machine, etc.

notes: can't run on cpu and gpu at the same time

In [13]:
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        result = sess.run(product)
        print result

[[ 12.]]


### Interactive Usage

In [29]:
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

print x
#print x.eval() # FailedPreconditionError: Attempting to use uninitialized value Variable
# initialize 'x' using the run() method of its initializer op
x.initializer.run()

# add op to subtract 'a' from 'x'
sub = tf.sub(x, a)
print sub.eval()
sess.close()

[-2. -1.]


### Variables

In [34]:
# create a variable that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# add one to the `state`.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# run `init` op to initialize variables after launching the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
    # Run the `init` op
    sess.run(init_op)
    print sess.run(state)
    
    # run the graph 3 times, each time `state is updated
    for _ in range(3):
        sess.run(update)
        print sess.run(state)
    
    

0
1
2
3


## Fetches
- we can fetch the value of multiple ops at once by passing them as a list to `sess.run()`
- All the ops needed to produce the values of the requested tensors are run once (not once per requested tensor).

In [41]:
input1 = tf.constant([3.0])
input2 = tf.constant([2.0])
input3 = tf.constant([5.0])
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

[array([ 21.], dtype=float32), array([ 7.], dtype=float32)]


## Feeds
- A feed temporarily replaces the output of an operation with a tensor value. You supply feed data as an argument to a run() call.

In [42]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
    print sess.run([output], feed_dict={input1:[7.], input2:[2.]})

[array([ 14.], dtype=float32)]
